In [1]:
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [2]:
!pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 29.7 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=fcd1479fb7dbe4d0b8a3b66dbb02595516bd6de283352e976b0ad65153bdf962
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [3]:
# Get Data
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data(one_hot=True)

Instructions for updating:
non-resource variables are not supported in the long term


100.0% 60276736 / 60270631


Succesfully downloaded 17flowers.tgz 60270631 bytes.
File Extracted
Starting to parse images...
Parsing Done!


In [4]:
x

array([[[[0.6627451 , 0.6039216 , 0.64705884],
         [0.6745098 , 0.6       , 0.6784314 ],
         [0.67058825, 0.5764706 , 0.69803923],
         ...,
         [0.01960784, 0.05490196, 0.        ],
         [0.01960784, 0.05098039, 0.00784314],
         [0.01960784, 0.05098039, 0.00784314]],

        [[0.627451  , 0.5882353 , 0.6666667 ],
         [0.62352943, 0.56078434, 0.68235296],
         [0.6       , 0.5058824 , 0.69411767],
         ...,
         [0.02352941, 0.05490196, 0.        ],
         [0.01960784, 0.05098039, 0.00784314],
         [0.01960784, 0.05098039, 0.00784314]],

        [[0.5764706 , 0.58431375, 0.56078434],
         [0.5686275 , 0.54509807, 0.5882353 ],
         [0.5294118 , 0.4627451 , 0.6       ],
         ...,
         [0.02745098, 0.04705882, 0.        ],
         [0.02352941, 0.05098039, 0.00784314],
         [0.02352941, 0.05490196, 0.01176471]],

        ...,

        [[0.07058824, 0.16470589, 0.03529412],
         [0.07058824, 0.16078432, 0.04705882]

In [5]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
x.shape

(1360, 224, 224, 3)

In [14]:
y.shape

(1360, 17)

In [18]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))

# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())



# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())


# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))


# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())


# Passing it to a dense layer
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.6))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_11 (Conv2D)          (None, 26, 26, 256)       614656    
                                                                 
 activation_1 (Activation)   (None, 26, 26, 256)      

In [19]:
# Compile 
opt = tf.optimizers.Adam(learning_rate = 0.05)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [20]:
# Train
model.fit(x, y, batch_size=84, epochs=5, verbose=1,validation_split=0.2, shuffle=True)

Train on 1088 samples, validate on 272 samples
Epoch 1/5
1088/1088 [==============================] - ETA: 0s - loss: 23.6651 - acc: 0.1241

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


1088/1088 [==============================] - 70s 64ms/sample - loss: 23.6651 - acc: 0.1241 - val_loss: 4437847.6213 - val_acc: 0.0956
Epoch 2/5
1088/1088 [==============================] - 68s 63ms/sample - loss: 18.0621 - acc: 0.1801 - val_loss: 219801.3408 - val_acc: 0.0956
Epoch 3/5
1088/1088 [==============================] - 68s 63ms/sample - loss: 14.3062 - acc: 0.2169 - val_loss: 227270.6985 - val_acc: 0.0956
Epoch 4/5
1088/1088 [==============================] - 69s 63ms/sample - loss: 12.8956 - acc: 0.1461 - val_loss: 101264.2991 - val_acc: 0.0478
Epoch 5/5
1088/1088 [==============================] - 69s 64ms/sample - loss: 10.8020 - acc: 0.1581 - val_loss: 112004.6928 - val_acc: 0.0478
